# 02: Model

In [1]:
%load_ext autoreload

%autoreload 2

Import libraries

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow_privacy.privacy.optimizers import dp_optimizer_vectorized
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.analysis.compute_noise_from_budget_lib import compute_noise

import wandb
from wandb.keras import WandbCallback


from synthesizers.cgan.model import (
    ConditionalGAN, 
    GANMonitor
)
from synthesizers.preprocessing.wesad import (
    WESADDataset, 
    LabelType
)
from synthesizers.utils.training import data_split



2023-10-09 13:34:04.730261: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Set parameters for training

In [3]:
SAMPLING_RATE = 1

# Training Hyperparameters
DP_TRAINING = False
NUM_FEATURES = 6
SEQ_LENGTH = 60
LATENT_DIM = SEQ_LENGTH
BATCH_SIZE = 8
HIDDEN_UNITS = 64
EPOCHS = 220 #todo 10
ACTIVATION = "relu"
RANDOM_SEED = 42
D_LEARNING_RATE = 0.0002 #todo 0.0002
G_LEARNING_RATE = 0.0001 #todo 0.0002
LOSS_FN = "binary_cross_entropy"
D_ARCHITECTURE = "lstm"
LOSO_TRAINING_WITHOUT_SUBJECT = False #todo "14"

# DP Training Hyperparameter
L2_NORM_CLIP = 1.0
NUM_MICROBATCHES = BATCH_SIZE
DP_LEARNING_RATE = 1e-3
DELTA = 1e-4


# Define run config
config = {
    "activation_function": ACTIVATION,
    "hidden_units": HIDDEN_UNITS,
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "random_seed": RANDOM_SEED,
    "num_features": NUM_FEATURES,
    "seq_length": SEQ_LENGTH,
    "dp_training": DP_TRAINING,
    "loss_function": LOSS_FN,
    "d_architecture": D_ARCHITECTURE,
}

if LOSO_TRAINING_WITHOUT_SUBJECT:
    config["WESAD_WITHOUT_SUBJ"] = LOSO_TRAINING_WITHOUT_SUBJECT

if DP_TRAINING:
    config["l2_norm_clip"] = L2_NORM_CLIP
    config["num_microbatches"] = NUM_MICROBATCHES
    config["dp_learning_rate"] = DP_LEARNING_RATE


Load data and split it into stress and non-stress for later testing

In [4]:
windows = np.load('data/wesad/wesad_windows.npy')
labels = np.load('data/wesad/wesad_labels.npy')

stress = windows[labels == 1]
amuse = windows[labels == 2]
base = windows[labels == 0]

windows = np.delete(windows, 6, axis=2)
stress = np.delete(stress, 6, axis=2)
amuse = np.delete(amuse, 6, axis=2)
base = np.delete(base, 6, axis=2)

num_split = 0.8
train_stress, test_stress = data_split(stress, num_split)
train_amuse, test_amuse = data_split(amuse, num_split)
train_base, test_base = data_split(base, num_split)

print(train_stress.shape)
print(test_stress.shape)
print(train_amuse.shape)
print(test_amuse.shape)
print(train_base.shape)
print(test_base.shape)

(258, 60, 6)
(65, 60, 6)
(143, 60, 6)
(36, 60, 6)
(464, 60, 6)
(117, 60, 6)


Calculate the noise for privacy guarantees

In [5]:
# get needed noise for target epsilon
min_noise = 1e-5
target_epsilons = [0.1, 1, 10]
noise_multipliers = {target_epsilon : compute_noise(
    windows.shape[0] // 2,
    BATCH_SIZE,
    target_epsilon,
    EPOCHS * 2,
    DELTA,
    min_noise
) for target_epsilon in target_epsilons}
print(noise_multipliers)

compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=windows.shape[0] // 2,
                                              batch_size=BATCH_SIZE,
                                              noise_multiplier=noise_multipliers[target_epsilons[0]],
                                              epochs=EPOCHS*2,
                                              delta=DELTA)

DP-SGD with sampling rate = 1.48% and noise_multiplier = 75.27182080353859 iterated over 29755 steps satisfies differential privacy with eps = 0.1 and delta = 0.0001.
DP-SGD with sampling rate = 1.48% and noise_multiplier = 8.987312249554396 iterated over 29755 steps satisfies differential privacy with eps = 1 and delta = 0.0001.
DP-SGD with sampling rate = 1.48% and noise_multiplier = 1.4188939763438704 iterated over 29755 steps satisfies differential privacy with eps = 10 and delta = 0.0001.
{0.1: 75.27182080353859, 1: 8.987312249554396, 10: 1.4188939763438704}
DP-SGD with sampling rate = 1.48% and noise_multiplier = 75.27182080353859 iterated over 29755 steps satisfies differential privacy with eps = 0.1 and delta = 0.0001.
The optimal RDP order is 128.0.


(0.09999999999999143, 128.0)

Create TF dataset from windows and labels

In [6]:
# Load dataset into tf dataset
dataset = tf.data.Dataset.from_tensor_slices((windows, labels))

# Shuffle, cache, and batch the dataset
dataset = dataset.shuffle(buffer_size=1024)
dataset = dataset.cache()
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)


tf.random.set_seed(RANDOM_SEED)
rand_train_stress = tf.random.normal(shape=(train_stress.shape[0], LATENT_DIM))

tf.random.set_seed(RANDOM_SEED)
rand_train_amuse = tf.random.normal(shape=(train_amuse.shape[0], LATENT_DIM))

tf.random.set_seed(RANDOM_SEED)
rand_train_base = tf.random.normal(shape=(train_base.shape[0], LATENT_DIM))

tf.random.set_seed(RANDOM_SEED)
rand_test_stress = tf.random.normal(shape=(test_stress.shape[0], LATENT_DIM))

tf.random.set_seed(RANDOM_SEED)
rand_test_amuse = tf.random.normal(shape=(test_amuse.shape[0], LATENT_DIM))

tf.random.set_seed(RANDOM_SEED)
rand_test_base = tf.random.normal(shape=(test_base.shape[0], LATENT_DIM))

Run Training in wandb environment

In [7]:
def main():
    cond_gan = ConditionalGAN(
        num_features=NUM_FEATURES,
        seq_length=SEQ_LENGTH,
        latent_dim=LATENT_DIM,
        discriminator=ConditionalGAN.conditional_discriminator(
            hidden_units=SEQ_LENGTH, 
            seq_length=SEQ_LENGTH, 
            num_features=NUM_FEATURES,
            filters=[32, 64, 32],
            activation_function= ACTIVATION,
            architecture=D_ARCHITECTURE, 
            #head_size=wandb.config.head_size#wandb.config.d_architecture
            #filters=[wandb.config.filter1, wandb.config.filter2, wandb.config.filter3],
            #kernel_sizes=[wandb.config.kernel_size1, wandb.config.kernel_size2, wandb.config.kernel_size3]
            ),
        generator=ConditionalGAN.conditional_generator(
            hidden_units=SEQ_LENGTH, 
            seq_length=SEQ_LENGTH, 
            latent_dim=LATENT_DIM,
            num_features=NUM_FEATURES,
            activation_function=ACTIVATION
        )
    )
    if DP_TRAINING:

        d_optimizer = dp_optimizer_vectorized.VectorizedDPAdamOptimizer( #vectorized adam am schnellsten
            l2_norm_clip=L2_NORM_CLIP,
            noise_multiplier=noise_multipliers[wandb.config.target_epsilon],
            num_microbatches=NUM_MICROBATCHES,
            learning_rate=DP_LEARNING_RATE
        )
    else:
        d_optimizer = Adam(learning_rate=D_LEARNING_RATE, beta_1=0.5) # get_optimizer(0.0002, wandb.config.optimizer)#

    g_optimizer = Adam(learning_rate=G_LEARNING_RATE, beta_1=0.5) # get_optimizer(0.0002, wandb.config.optimizer)#

    cond_gan.compile(
        d_optimizer= d_optimizer, # Adam(learning_rate=0.0002, beta_1=0.5),
        g_optimizer= g_optimizer, # Adam(learning_rate=0.0002, beta_1=0.5), #optimizer
        loss_fn= tf.keras.losses.BinaryCrossentropy(from_logits=True),
    )

    print(f"{cond_gan.d_optimizer} is used")

    if DP_TRAINING:
        generator_save_path = f"models/dp/dpcgan3er/"
    else:
        generator_save_path = f"models/no_dp/cgan3er/"

    history = cond_gan.fit(
        dataset,
        epochs=EPOCHS,
        callbacks=[
            # GANMonitor(
            #     train_stress,
            #     train_amuse,
            #     train_base,
            #     test_stress,
            #     test_amuse,
            #     test_base,
            #     rand_train_stress,
            #     rand_train_amuse,
            #     rand_train_base,
            #     rand_test_stress,
            #     rand_test_amuse,
            #     rand_test_base,
            #     num_seq=50,
            #     save_path=generator_save_path,
            #     batch_size=BATCH_SIZE,
            #     seq_length=SEQ_LENGTH,
            #     num_features=NUM_FEATURES,
            #     dp=DP_TRAINING,
            # )
        ],
    )

    if DP_TRAINING:
        base_path = f"models/dp/{wandb.run.name}/"
        cond_gan.generator.save(f"{base_path}cgan_generator")
        cond_gan.discriminator.save(f"{base_path}cgan_discriminator")
    elif LOSO_TRAINING_WITHOUT_SUBJECT:
        base_path = f"models/no_dp/loso/sub{LOSO_TRAINING_WITHOUT_SUBJECT}/{wandb.run.name}/"
        cond_gan.generator.save(f"{base_path}cgan_generator")
        cond_gan.discriminator.save(f"{base_path}cgan_discriminator")
    else:
        base_path = f"models/no_dp/cgan3er/"
        cond_gan.generator.save(f"{base_path}cgan_generator")
        cond_gan.discriminator.save(f"{base_path}cgan_discriminator")

In [8]:
main()

discriminator_architecture:  lstm


2023-10-09 13:34:20.417593: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-09 13:34:20.419394: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-09 13:34:20.421223: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

<keras.optimizers.legacy.adam.Adam object at 0x13de700a0> is used
Epoch 1/220


2023-10-09 13:34:21.339728: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-09 13:34:21.342069: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-09 13:34:21.343767: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

136/136 [==============================] - 30s 91ms/step - g_loss: 0.8323 - d_loss: 0.4748 - div_term: 0.2071
Epoch 2/220
136/136 [==============================] - 12s 89ms/step - g_loss: 3.5794 - d_loss: 0.0536 - div_term: 0.4607
Epoch 3/220
136/136 [==============================] - 12s 89ms/step - g_loss: 3.7357 - d_loss: 0.0506 - div_term: 0.9064
Epoch 4/220
136/136 [==============================] - 12s 90ms/step - g_loss: 1.2713 - d_loss: 0.3189 - div_term: 1.2808
Epoch 5/220
136/136 [==============================] - 12s 89ms/step - g_loss: -0.1891 - d_loss: 0.6513 - div_term: 1.2494
Epoch 6/220
136/136 [==============================] - 14s 100ms/step - g_loss: 0.7347 - d_loss: 0.4116 - div_term: 1.0858
Epoch 7/220
136/136 [==============================] - 12s 90ms/step - g_loss: 0.5317 - d_loss: 0.3205 - div_term: 1.4986
Epoch 8/220
136/136 [==============================] - 13s 98ms/step - g_loss: -0.5303 - d_loss: 0.4259 - div_term: 2.3291
Epoch 9/220
136/136 [============

...

INFO:tensorflow:Assets written to: models/no_dp/cgan3er/cgan_generator/assets


INFO:tensorflow:Assets written to: models/no_dp/cgan3er/cgan_generator/assets


2023-10-09 14:20:37.733688: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-09 14:20:37.735872: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-09 14:20:37.737464: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

INFO:tensorflow:Assets written to: models/no_dp/cgan3er/cgan_discriminator/assets


INFO:tensorflow:Assets written to: models/no_dp/cgan3er/cgan_discriminator/assets


In [9]:
import os
os.system("say 'C GAN IST FERTIG'")

0